In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For customizing our plots.

from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
train =pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/train.csv')
test =pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/test.csv')
sample_submission =pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')

In [ ]:
train.head()

Language Count 

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(10, 4), dpi=100)
sns.countplot(train['language'], ax=axes[0])
axes[1].pie(train['language'].value_counts(),
            labels=['Hindi', 'Tamil'],
            autopct='%1.2f%%',
            shadow=True,
            explode=(0.05, 0),
            startangle=60)
fig.suptitle('Distribution of the language', fontsize=24)
plt.show()

In [ ]:
train['num_words_context'] = train.context.map(lambda x : len(x.split()))
train['num_words_question'] = train.question.map(lambda x : len(x.split()))
train['num_words_answer'] = train.answer_text.map(lambda x : len(x.split()))


train_hi = train[train.language == 'hindi']
train_ta = train[train.language == 'tamil']

## Context Lengths
#### Let's start with the number of characters in context, question and answer.

In [ ]:
def plot_hist(data : pd.Series, title : str):
    fig = plt.figure(constrained_layout=True, figsize=(18, 8))
    grid = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)

    ax1 = fig.add_subplot(grid[0, :2])
    # Set the title.
    ax1.set_title(title)
    # plot the histogram.
    sns.distplot(data,
                  hist=True,
                  kde=True,
                  ax=ax1,
                  color='#e74c3c')
    ax1.set(ylabel='Frequency')
    ax1.set(xlabel='Character Word Count')
    ax1.xaxis.set_major_locator(MaxNLocator(nbins=20))
    return plt


In [ ]:
title = 'Word Count Per "Context"'
plt = plot_hist(train_hi['num_words_context'], "Hindi")
plt.title(f'{title}', fontsize=14)
plot_hist(train_ta['num_words_context'], "Tamil")


In [ ]:
title = 'Word Count Per "Question"'
plt = plot_hist(train_hi['num_words_question'], "Hindi")
plt.title(f'{title}', fontsize=14)
plot_hist(train_ta['num_words_question'], "Tamil")


In [ ]:
title = 'Word Count Per "Answer"'
plt = plot_hist(train_hi['num_words_answer'], "Hindi")
plt.title(f'{title}', fontsize=14)
plot_hist(train_ta['num_words_answer'], "Tamil")

### Length of Context, question and answer

In [ ]:
def plot_word_number_histogram(data_hi, data_ta, title):
    
    """A function for comparing word counts"""

    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(18, 6), sharey=True)
    sns.distplot(data_hi.str.split().map(lambda x: len(x)), ax=axes[0], color='#e74c3c')
    sns.distplot(data_ta.str.split().map(lambda x: len(x)), ax=axes[1], color='#e74c3c')
    
    axes[0].set_xlabel('Word Count')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Tamil')
    axes[1].set_xlabel('Word Count')
    axes[1].set_title('Hindi')
    
    fig.suptitle(title, fontsize=24, va='baseline')
    
    fig.tight_layout()

In [ ]:
plot_word_number_histogram( train_hi.context, train_ta.context, 'Words length in Context')
plot_word_number_histogram( train_hi.question, train_ta.question, 'Words length in Question')
plot_word_number_histogram( train_hi.answer_text, train_ta.answer_text, 'Words length in Answer')


### Word Cloud

In [ ]:
hindi_stopword = ['अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'अंदर', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर', 'करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नके', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वग़ैरह', 'वर्ग', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'संग', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने']

In [ ]:
def plot_wordcloud(text, title, title_size, stopword, font_path):
    """ A function for creating wordcloud images """
    allwords = ''.join(text.to_list())
    wordcloud = WordCloud(
        width=1200,
        height=800,
        font_path=font_path,
        background_color='black',
        stopwords=set(stopword),
        max_words=150,
        scale=3,
        contour_width=0.1,
        contour_color='grey',
    ).generate(str(allwords))
    fig = plt.figure(figsize=(18, 18), facecolor='white')
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title(title,
              fontdict={
                  'size': title_size,
                  'verticalalignment': 'bottom'
              })
    plt.tight_layout(pad=0)
    plt.show()   

In [ ]:
plot_wordcloud(train_hi.context, "Word Cloud Hindi", 15, hindi_stopword, '/kaggle/input/devanagari/Lohit-Devanagari.ttf')